In [26]:
import pickle
from collections import defaultdict
import pandas as pd

In [27]:
with open('./feedback.pkl', 'rb') as f:
    feedback = pickle.load(f)

In [28]:
print(type(feedback))
relevant_fields = [
    'SixForceValue',
    'ActualTCPForce',
    'TimeStamp',
    'TargetQuaternion',
    'ActualQuaternion',
    'QTarget',
    'QActual',
    'QDTarget',
    'QDActual',
    'CurrentCommandId',
    'RobotMode',
]
for x in relevant_fields:
    print(x, feedback[x][0])

<class 'numpy.ndarray'>
SixForceValue [0. 0. 0. 0. 0. 0.]
ActualTCPForce [0. 0. 0. 0. 0. 0.]
TimeStamp 1747322366638
TargetQuaternion [ 5.55111512e-17  5.55111512e-17  7.07106769e-01 -7.07106769e-01]
ActualQuaternion [ 4.48877017e-06  2.11853167e-05  7.07106769e-01 -7.07106769e-01]
QTarget [0. 0. 0. 0. 0. 0.]
QActual [ 2.03954114e-05  0.00000000e+00  2.03954114e-05  1.37329102e-03
 -1.37329102e-03  6.86645508e-04]
QDTarget [0. 0. 0. 0. 0. 0.]
QDActual [ 0.02730266 -0.02548747 -0.01919208  1.82304382 -0.79925537  3.13316345]
CurrentCommandId 4
RobotMode 5


read feedback data

In [29]:
def read_buffered_pickle(filename='./feedback_all.pkl'):
    all_data = []
    try:
        with open(filename, 'rb') as f:
            while True:
                batch = pickle.load(f)
                all_data.extend(batch)
    except EOFError:
        pass
    return all_data

feedback = read_buffered_pickle()
print(f"Total data points loaded: {len(feedback)}")

Total data points loaded: 10600


read (command id, path) pairs

In [30]:
with open('./command_path_mapping.pkl', 'rb') as f:
    command_path_mapping = pickle.load(f)

pandas

In [31]:
df = pd.DataFrame(feedback)

data wrangling

In [33]:
df['path_id'] = df['CurrentCommandId'].map(command_path_mapping)
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
df_sorted = df.sort_values(by=['path_id', 'TimeStamp'], ascending=True)

In [35]:
print(df_sorted)
df_sorted.to_csv('./feedback.csv')

                        SixForceValue                  ActualTCPForce  \
90     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
91     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
92     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
93     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
94     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
...                               ...                             ...   
10595  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
10596  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
10597  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
10598  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
10599  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]   

                          TimeStamp  \
90    1970-01-01 00:29:07.335810707   
91    1970-01-01 00:29:07.335810716   
92    